In [1]:
import pandas as pd
import numpy as np

from collections import deque
from uuid import uuid4

In [2]:
POSITIONS = ["QB", "RB", "WR", "TE"]

In [63]:
from pyomo.opt import SolverFactory
from pyomo.environ import (
    AbstractModel, 
    Set, 
    Param, 
    Boolean, 
    Objective, 
    Constraint, 
    Var, 
    summation, 
    maximize,
    inequality,
    value,
)
from pyomo.core.expr.current import AbsExpression

def basic_model(budget=55000, max_per_team=None):
    model = AbstractModel()

    model.players = Set()
    model.teams = Set()

    model.prices = Param(model.players)
    model.points = Param(model.players)
    model.positions = Param(model.players, range(len(POSITIONS)))
    model.player_teams = Param(model.players, model.teams)

    model.x = Var(model.players, domain=Boolean)

    def obj_expression(model):
        return summation(model.points, model.x)

    model.OBJ = Objective(rule=obj_expression, sense=maximize)

    def budget_constraint(model):
        return sum(model.prices[i] * model.x[i] for i in model.players) <= budget

    def qb_constraint(model):
        return sum(model.positions[i, 0] * model.x[i] for i in model.players) == 1

    def rb_constraint(model):
        return inequality(2, sum(model.positions[i, 1] * model.x[i] for i in model.players), 3)

    def wr_constraint(model):
        return inequality(3, sum(model.positions[i, 2] * model.x[i] for i in model.players), 4)

    def te_constraint(model):
        return inequality(1, sum(model.positions[i, 3] * model.x[i] for i in model.players), 2)

    def flex_constraint(model):
        return sum(model.positions[i, p] * model.x[i] for i in model.players for p in range(1, 4)) == 7

    def flex_constraint(model):
        return sum(model.positions[i, p] * model.x[i] for i in model.players for p in range(1, 4)) == 7

    def team_constraint(model, t):
        return sum(model.player_teams[i, t] * model.x[i] for i in model.players) <= max_per_team
    
    model.BudgetConstraint = Constraint(rule=budget_constraint)
    model.QBConstraint = Constraint(rule=qb_constraint)
    model.RBConstraint = Constraint(rule=rb_constraint)
    model.WRConstraint = Constraint(rule=wr_constraint)
    model.TEConstraint = Constraint(rule=te_constraint)
    model.FlexConstraint = Constraint(rule=flex_constraint)
    
    if max_per_team is not None:
        model.TeamConstraint = Constraint(model.teams, rule=team_constraint)
    
    return model

def single_game_model(budget=60000):
    model = AbstractModel()

    model.players = Set()
    model.teams = Set()

    model.prices = Param(model.players)
    model.points = Param(model.players)
    model.positions = Param(model.players, range(len(POSITIONS)))
    model.player_teams = Param(model.players, model.teams)

    model.x = Var(model.players, domain=Boolean)

    def obj_expression(model):
        return summation(model.points, model.x)

    model.OBJ = Objective(rule=obj_expression, sense=maximize)

    def budget_constraint(model):
        return sum(model.prices[i] * model.x[i] for i in model.players) <= budget
    
    def player_constraint(model):
        return sum(model.x[i] for i in model.players) == 5

    model.BudgetConstraint = Constraint(rule=budget_constraint)
    model.PlayerConstraint = Constraint(rule=player_constraint)
    
    return model

# Real Deal Holyfield

In [4]:
df = pd.read_csv("../data/draftkings/slates/fd_slate_and_preds.csv")

In [31]:
df

,player_id,season,week,name,pts,pts_max,pts_min,expected_start_date,total_spots,entry_fee,guaranteed_prizes,tournament_name,position,salary,current_team_id,franchise_id,team_name,rank,game_time,std
0,27,2020,2,Ben Roethlisberger,17.662,23.9742,13.4984,2020-09-20 17:00:00.000000,55556,3.0,150000.0,$150K Sun NFL Huddle (Experienced Players Excl...,QB,7500,pit2020,PIT,Pittsburgh Steelers,1.0,2020-09-20 13:00:00 EDT,5.2379
1,27,2020,2,Ben Roethlisberger,17.662,23.9742,13.4984,2020-09-20 17:00:00.000000,892857,4.0,3000000.0,$3M NFL Sunday Million ($1M to 1st),QB,7500,pit2020,PIT,Pittsburgh Steelers,1.0,2020-09-20 13:00:00 EDT,5.2379
2,27,2020,2,Ben Roethlisberger,17.662,23.9742,13.4984,2020-09-20 17:00:00.000000,26455,9.0,200000.0,$200K Sun NFL Rush ($40K to 1st),QB,7500,pit2020,PIT,Pittsburgh Steelers,1.0,2020-09-20 13:00:00 EDT,5.2379
3,27,2020,2,Ben Roethlisberger,17.662,23.9742,13.4984,2020-09-20 17:00:00.000000,16042,22.0,300000.0,$300K Sun NFL Kickoff ($100K to 1st),QB,7500,pit2020,PIT,Pittsburgh Steelers,1.0,2020-09-20 13:00:00 EDT,5.2379
4,27,2020,2,Ben Roethlisberger,17.662,23.9742,13.4984,2020-09-20 17:00:00.000000,13368,44.0,500000.0,$500K Sun NFL Bomb ($100K to 1st),QB,7500,pit2020,PIT,Pittsburgh Steelers,1.0,2020-09-20 13:00:00 EDT,5.2379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5971,27803,2020,2,Malik Taylor,0.007,6.9520,0.0000,2020-09-20 17:00:00.000000,39682,9.0,300000.0,$300K Sun NFL Rush ($100K to 1st),WR,4500,gnb2020,GNB,Green Bay Packers,2.0,2020-09-20 13:00:00 EDT,3.4760
5972,27803,2020,2,Malik Taylor,0.007,6.9520,0.0000,2020-09-20 17:00:00.000000,1426,33.0,40000.0,$40K Sun NFL Bomb ($10K to 1st),WR,4500,gnb2020,GNB,Green Bay Packers,2.0,2020-09-20 13:00:00 EDT,3.4760
5973,27803,2020,2,Malik Taylor,0.007,6.9520,0.0000,2020-09-20 17:00:00.000000,20,0.0,0.0,20-Player League ($0),WR,4500,gnb2020,GNB,Green Bay Packers,2.0,2020-09-20 13:00:00 EDT,3.4760
5974,27803,2020,2,Malik Taylor,0.007,6.9520,0.0000,2020-09-20 17:00:00.000000,20,0.0,0.0,20-Player League ($0),WR,4500,gnb2020,GNB,Green Bay Packers,2.0,2020-09-20 13:00:00 EDT,3.4760


In [7]:
df["std"] = ((df["pts_max"] - df["pts"]) + (df["pts"] - df["pts_min"])) / 2

In [8]:
df.tournament_name.value_counts()

20-Player League ($0)                                              3099
$500K Sun NFL Bomb ($100K to 1st)                                   339
$150K Sun NFL Huddle (Experienced Players Excluded)                 339
$200K Sun NFL Rush ($40K to 1st)                                    339
$40K Sun NFL Bomb ($10K to 1st)                                     339
$2M WFFC Qualifier #9                                               339
$300K Sun NFL Kickoff ($100K to 1st)                                339
$3M NFL Sunday Million ($1M to 1st)                                 339
$300K Sun NFL Rush ($100K to 1st)                                   262
$350K Sun NFL Rush ($100K to 1st)                                    77
$20K Thu NFL 2nd Half Special ($4K to 1st)                           28
$100K Thu NFL Bomb ($20K to 1st)                                     28
$1.2M NFL Thursday Million ($250K to 1st + $100K Last TD Bonus)      28
$2M World Fantasy Football Championship Qualifier #8            

In [19]:
tournament_df = df[df["tournament_name"] == "$300K Sun NFL Kickoff ($100K to 1st)"]
tournament_df = tournament_df[(tournament_df["position"] != "QB") | (tournament_df["rank"] == 1)]

In [20]:
tournament_df.shape

(311, 20)

In [21]:
tournament_df.team_name.unique()

array(['Pittsburgh Steelers', 'Atlanta Falcons', 'Chicago Bears',
       'Tampa Bay Buccaneers', 'Philadelphia Eagles', 'Detroit Lions',
       'Green Bay Packers', 'Arizona Cardinals', 'New York Jets',
       'Indianapolis Colts', 'Miami Dolphins', 'New York Giants',
       'Buffalo Bills', 'Houston Texans', 'Baltimore Ravens',
       'Kansas City Chiefs', 'Minnesota Vikings', 'San Francisco 49ers',
       'Los Angeles Rams', 'Jacksonville Jaguars', 'Los Angeles Chargers',
       'Tennessee Titans', 'Carolina Panthers',
       'Washington Football Team', 'Denver Broncos', 'Dallas Cowboys'],
      dtype=object)

In [22]:
tournament_df.position.value_counts()

WR    125
RB     85
TE     77
QB     24
Name: position, dtype: int64

In [87]:
team_names = tournament_df.team_name.unique()
lineups = []
injuries = []
for i in range(20):
    player_pool = tournament_df[(~tournament_df.name.isin(injuries))]
    player_prices = {i: price for i, price in enumerate(player_pool.salary.tolist())}
    points = tournament_df.apply(lambda x: np.random.normal(x["pts"], x["std"]), axis="columns")
    player_points = {i: points for i, points in enumerate(points.tolist())}
    positions = pd.get_dummies(player_pool.position.astype(pd.CategoricalDtype(categories=POSITIONS))).values
    positions = {(i, p): positions[i, p] 
                 for i in range(positions.shape[0]) 
                 for p in range(positions.shape[1])}
    players = list(range(len(player_prices)))
    teams = list(range(len(team_names)))
    
    player_teams = pd.get_dummies(player_pool.team_name.astype(pd.CategoricalDtype(categories=team_names))).values
    player_teams = {(i, t): player_teams[i, t] 
                 for i in players 
                 for t in teams}

    data = {
        None: {
            "players": {None: players},
            "prices": player_prices,
            "points": player_points,
            "positions": positions,
            "teams": teams,
            "player_teams": player_teams,
        }, 
    }
    instance = basic_model(budget=57000, max_per_team=2).create_instance(data=data)
    opt = SolverFactory('glpk')
    optimizer_result = opt.solve(instance)
    lineup = player_pool[[value > 0 for value in instance.x.get_values().values()]]    
    lineup["lineup_id"] = uuid4().hex
    lineups.append(lineup)

<ipython-input-87-7ce47c2abc4c>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lineup["lineup_id"] = uuid4().hex


In [88]:
lineups = pd.concat(lineups, ignore_index=True)

In [90]:
with pd.option_context("display.max_rows", None):
    display(lineups[["lineup_id", "name", "position", "team_name", "pts", "rank"]])

,lineup_id,name,position,team_name,pts,rank
0,7334061ededd4508a59656a941432406,DeAndre Hopkins,WR,Arizona Cardinals,14.7120,1.0
1,7334061ededd4508a59656a941432406,Hunter Henry,TE,Los Angeles Chargers,8.6900,1.0
2,7334061ededd4508a59656a941432406,Kenyan Drake,RB,Arizona Cardinals,13.6010,1.0
3,7334061ededd4508a59656a941432406,Christian McCaffrey,RB,Carolina Panthers,15.4220,1.0
4,7334061ededd4508a59656a941432406,Curtis Samuel,WR,Carolina Panthers,8.6690,1.0
5,7334061ededd4508a59656a941432406,Sam Darnold,QB,New York Jets,13.9326,1.0
6,7334061ededd4508a59656a941432406,Allen Lazard,WR,Green Bay Packers,9.6750,1.0
7,7334061ededd4508a59656a941432406,Devin Singletary,RB,Buffalo Bills,13.8210,1.0
8,1e18742044554afd8c9ba9a284fe9708,Danny Amendola,WR,Detroit Lions,10.3250,2.0
9,1e18742044554afd8c9ba9a284fe9708,Julio Jones,WR,Atlanta Falcons,18.5140,1.0


In [89]:
lineups.to_csv("../data/draftkings/lineups/20200920_57k.csv", index=False)